### Imports and Constants

In [1]:
import numpy as np
import cv2
import os
from PIL import Image
import pickle
import pyautogui

VIDEONAME = "LiveFaceRecognition.mp4"
FRAMES_PER_SECOND = 24.0
RESOLUTION = "720p"

### Video Settings

In [2]:
def change_res(cap, width, height):
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# dictionary of the standard dimensions for the resolution
STANDARD_DIMENSIONS = {
    "480p": (640, 480),
    "720p": (1280, 720)
}

def get_dims(cap, res):
    width, height = STANDARD_DIMENSIONS["480p"]
    if res in STANDARD_DIMENSIONS:
        width, height = STANDARD_DIMENSIONS[res]
    change_res(cap, width, height)
    
    return width, height

# dictoniary of the videotypes
VIDEO_TYPE = {
    "avi": cv2.VideoWriter_fourcc(*'XVID'),
    "mp4": cv2.VideoWriter_fourcc(*'XVID')
}

def get_video_type(filename):
    filename, ext = os.path.splitext(filename)
    if ext in VIDEO_TYPE:
        return VIDEO_TYPE[ext]
    return VIDEO_TYPE["mp4"]

### Mouse Movement Function

In [3]:
def MoveMouse(XPos, YPos):
    
    xPos = XPos
    
    TopBarDelayPx = 31 # that bar with the name of the program at the top
    yPos = YPos + TopBarDelayPx
    
    pyautogui.moveTo(XPos, YPos)

## The Actual Logic

In [4]:
face_cascade = cv2.CascadeClassifier('data/cascades/haarcascade_frontalface_alt.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

with open("labels.pickle", "rb") as f:
    original_labels = pickle.load(f)
    labels = {v: k for k, v in original_labels.items()}
cap = cv2.VideoCapture(0)



# Changes video resolution
dims = get_dims(cap, RESOLUTION)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # need gray to make face_cascade work
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5, minNeighbors = 5)
    
    # face recognizer
    for (x, y, w, h) in faces:
        
        # region of interest: the part which shows your face
        roi_gray = gray[y: y+h, x: x+w]
        roi_color = frame[y: y+h, x: x+w]
        
        # saves the img
        gray_img_item = "my-gray-image.png"
        colored_img_item = "my-colored-image.png"
        cv2.imwrite(gray_img_item, roi_gray)
        cv2.imwrite(colored_img_item, roi_color)
                    
        # Draws box if face is seen
        blue_color = (0, 255, 0) # BGR scale
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), blue_color, stroke)        
        
        # Moves mouse to the face position         
        MousePosX = x + w/2
        MousePosY = y + h/2
        MoveMouse(MousePosX, MousePosY)
    
    if ret == True:
        
        # Display the resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
# When everything done, release everything
cap.release()
cv2.destroyAllWindows()

# Train Faces

In [25]:
BASE_DIR = os.path.dirname(os.path.abspath(""))
image_dir = os.path.join(BASE_DIR, "live_face_recognition\data\images")

face_cascade = cv2.CascadeClassifier('data/cascades/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).lower()
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            
            pil_image = Image.open(path).convert("L") # greyscales the img
            image_array = np.array(pil_image, "uint8")
            faces = face_cascade.detectMultiScale(image_array, scaleFactor = 1.5, minNeighbors = 5)
            
            for (x, y, w, h) in faces:
                roi = image_array[y: y+h, x: x+w]
                x_train.append(roi)
                y_labels.append(id_)

with open("labels.pickle", "wb") as f:
    pickle.dump(label_ids, f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")